# Concatenating Tables with Set-Like Operations

One of the two way of combining two tables is to stack one table on top of the other.  When stacking two tables on top of one another, we need to decide

1. If we combine columns based on position or name (and if combining by name, what do we do with mismatches?)
2. How to decide which rows to keep.  In this case, we will take some guidance from SQL clauses.

## Three Types of Operations

* **Union:** Keeps rows from either table.
* **Intersection:** Only keeps common columns
* **Set Difference/Except:** Keep rows from the left table *except* those in the right table.

## Set Operations in Action 

<img src="./img/table_verbs_set.gif" width=800>

## All Operations Match by Position

All operations

* Match columns by position
* Require same number/type of columns

## Distinct Versus All

* **UNION/INTERSECT/SET DIFFERENE** are **DISTINCT**
    * Only keeps distinct rows, removing duplicates.
* **UNION ALL/INTERSECT ALL/SET DIFFERENCE ALL**
    * Keeps duplicate rows

In [1]:
import pandas as pd
from dfply import *

In [2]:
sales_may = pd.read_csv('./data/auto_sales_may.csv')
sales_may

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9


In [3]:
sales_apr = pd.read_csv('./data/auto_sales_apr.csv')
sales_apr

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


## Unions with `dfply`

Use `left_table >> union(right_table)`

In [4]:
sales_may >> union(sales_apr)

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


## `dfply.union` is distinct

Since Ann have the same sales each month, her row only included one row.  Note that we can use `keep='last'` to `keep='first'` to determine which row is kept.

In [5]:
sales_may >> union(sales_apr, keep='last')

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


In [6]:
sales_may >> union(sales_apr, keep='first')

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


## Making `union_all`

We can use `pd.concat` to perform a `UNION ALL`

In [7]:
from more_dfply import union_all
sales_may >> union_all(sales_apr)

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9
4,0,Ann,22,18,15,12
5,1,Bob,19,12,17,20
6,2,Yolanda,19,8,32,15
7,3,Xerxes,12,23,18,9


## Adding a month column

Another way to keep both of Ann's sales rows is adding a month column (which we should probably do anyway).

In [8]:
sales_may >> mutate(month = 'May') >> union(sales_apr >> mutate(month = 'April'))

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck,month
0,0,Ann,22,18,15,12,May
1,1,Bob,20,14,6,24,May
2,2,Yolanda,19,10,28,17,May
3,3,Xerxes,11,27,17,9,May
0,0,Ann,22,18,15,12,April
1,1,Bob,19,12,17,20,April
2,2,Yolanda,19,8,32,15,April
3,3,Xerxes,12,23,18,9,April


## Finding common rows with `dfply.intersect`

In [9]:
sales_may >> intersect(sales_apr)

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12


## Finding rows unique to the left table.

Use `left_table >> dfply.set_diff(right_table)`

In [10]:
sales_may >> set_diff(sales_apr)

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9


## <font color="red"> Exercise 1 </font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Use `glob` to get all 6 file paths.
2. Use a regular expression to create a `lambda` function that pulls the month from the files.
3. Read the 6 data frames into a `dict` with keys equal to the month name and values containing the corresponding data frame.
4. Write a helper function that adds a month column each dictionary.  Use a dictionary comprehension to apply this helper to each `df`.
5. Use the accumulator pattern and `dfply.union` to combine these 6 data frames into one combined `df`
6. Inspect the head and shape of the resulting `df`

In [66]:
# Your code here
import glob
paths = glob.glob('./data/uber-raw-data-???14-sample.csv')

In [67]:
paths

['./data\\uber-raw-data-apr14-sample.csv',
 './data\\uber-raw-data-aug14-sample.csv',
 './data\\uber-raw-data-jul14-sample.csv',
 './data\\uber-raw-data-jun14-sample.csv',
 './data\\uber-raw-data-may14-sample.csv',
 './data\\uber-raw-data-sep14-sample.csv']

In [65]:
import re
pattern = "\./data\\uber-raw-data-apr14-sample.csv"
regex = re.compile(pattern)

error: incomplete escape \ube at position 7

In [54]:
for path in paths:
    print(regex.search(path))

None
None
None
None
None
None
